<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -qU kaggle pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 93.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.


In [14]:
import os
import json
import sys
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from kaggle.api.kaggle_api_extended import KaggleApi

In [15]:
def ensure_directory(directory):
    """Create directory if it doesn't exist"""
    Path(directory).mkdir(parents=True, exist_ok=True)
    return directory

In [16]:
def setup_kaggle_credentials():
    """Set up Kaggle credentials with validation"""
    print("\n🔐 Kaggle API Setup")
    kaggle_path = Path.home() / '.kaggle'
    ensure_directory(kaggle_path)

    credentials = {}
    while True:
        credentials['username'] = input("Enter Kaggle username: ").strip()
        credentials['key'] = input("Enter Kaggle API key: ").strip()

        # Save credentials
        with open(kaggle_path / 'kaggle.json', 'w') as f:
            json.dump(credentials, f)
        os.chmod(kaggle_path / 'kaggle.json', 0o600)

        # Test credentials
        try:
            api = KaggleApi()
            api.authenticate()
            print("✅ Credentials validated successfully!")
            return api
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            if not input("Retry credentials? (y/n): ").lower().startswith('y'):
                sys.exit(1)

In [17]:
def accept_terms_reminder():
    """Show reminder to accept competition terms"""
    print("\n📝 Legal Requirement:")
    print("Before downloading, you MUST accept the dataset terms:")
    print("1. Visit: https://www.kaggle.com/facebook/dfdc-faceforensics-preview")
    print("2. Click 'Join Competition' (if required)")
    print("3. Read and accept the competition rules")
    input("\nPress Enter AFTER completing these steps to continue...")

In [18]:
def download_dataset(api, output_dir):
    """Download and organize DFDC dataset"""
    try:
        print(f"\n📥 Downloading DFDC FaceForensics Preview to {output_dir}...")
        api.dataset_download_files(
            'facebook/dfdc-faceforensics-preview',
            path=output_dir,
            unzip=True,
            quiet=False
        )

        # Organize files
        print("\n🗂 Organizing dataset structure...")
        dfdc_dir = Path(output_dir) / 'dfdc-faceforensics-preview'
        if not dfdc_dir.exists():
            raise FileNotFoundError("Dataset directory not found after download")

        return dfdc_dir
    except Exception as e:
        print(f"\n❌ Download failed: {str(e)}")
        print("Possible solutions:")
        print("- Ensure you accepted terms on Kaggle")
        print("- Check your internet connection")
        print("- Verify available disk space")
        sys.exit(1)

In [19]:
def process_metadata(dataset_dir):
    """Process dataset metadata and create summary"""
    print("\n📊 Processing metadata...")
    try:
        metadata_path = dataset_dir / 'metadata.json'

        if not metadata_path.exists():
            raise FileNotFoundError("Metadata file not found")

        with open(metadata_path) as f:
            metadata = json.load(f)

        # Create dataframe with validation
        df = pd.DataFrame.from_dict(metadata, orient='index').reset_index()
        df.columns = ['filename', 'label', 'original', 'split']

        # Add additional metadata
        df['is_train'] = df['split'] == 'train'
        df['is_test'] = df['split'] == 'test'

        # Save summaries
        summary_path = dataset_dir / 'dataset_summary.csv'
        df.to_csv(summary_path, index=False)

        print(f"✅ Metadata processed ({len(df)} entries)")
        print(df.label.value_counts().to_string())

        return df
    except Exception as e:
        print(f"⚠️ Metadata processing skipped: {str(e)}")
        return None

In [20]:
def main():
    print("\n🎬 DFDC FaceForensics Preview Downloader")
    print("==========================================")

    # Setup API
    api = setup_kaggle_credentials()

    # Check terms acceptance
    accept_terms_reminder()

    # Set output directory
    output_dir = ensure_directory(Path.cwd() / 'dfdc_dataset')

    # Download dataset
    dataset_dir = download_dataset(api, output_dir)

    # Process metadata
    metadata_df = process_metadata(dataset_dir)

    # Final report
    print("\n✅ Setup Complete")
    print(f"Dataset location: {dataset_dir}")
    if metadata_df is not None:
        print("\nDataset Summary:")
        print(f"Total Videos: {len(metadata_df)}")
        print(f"Fake Videos: {len(metadata_df[metadata_df['label'] == 'FAKE'])}")
        print(f"Real Videos: {len(metadata_df[metadata_df['label'] == 'REAL'])}")

    print("\nNext steps:")
    print("1. Review the dataset_summary.csv file")
    print("2. Explore the video files in the dataset directory")
    print("3. Begin your deepfake analysis!")

if __name__ == "__main__":
    main()


🎬 DFDC FaceForensics Preview Downloader

🔐 Kaggle API Setup
Enter Kaggle username: kaif8755
Enter Kaggle API key: 551a523b812b9721fc1d3ad5add8f667
✅ Credentials validated successfully!

📝 Legal Requirement:
Before downloading, you MUST accept the dataset terms:
1. Visit: https://www.kaggle.com/facebook/dfdc-faceforensics-preview
2. Click 'Join Competition' (if required)
3. Read and accept the competition rules

Press Enter AFTER completing these steps to continue...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




📥 Downloading DFDC FaceForensics Preview to /content/dfdc_dataset...
Dataset URL: https://www.kaggle.com/datasets/facebook/dfdc-faceforensics-preview

❌ Download failed: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Wed, 29 Jan 2025 08:35:23 GMT', 'Access-Control-Allow-Credentials': 'true', 'Access-Control-Allow-Origin': '*', 'Set-Cookie': 'ka_sessionid=8e38768d5a532333ae3b5c2f1f92a2ca; max-age=2626560; path=/, GCLB=CLKoguipo567FhAD; path=/; HttpOnly', 'Vary': 'Accept-Encoding', 'X-Kaggle-MillisecondsElapsed': '9', 'X-Kaggle-RequestId': 'bfec9dd7ea5cc046355a99d31df4d894', 'X-Kaggle-ApiVersion': '1.6.17', 'X-Kaggle-HubVersion': '0.3.6', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=63072000; includeSubDomains; preload', 'Content-Security-Policy': "object-src 'none'; script-src 'nonce-oMbKDNe2w4Uv2ioci/yvxA==' 'report-sample' 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:; base-uri 'none'

TypeError: object of type 'NoneType' has no len()